# Random Forest

##### This project implements a random forest model to predict (classification) whether or not a person will default on their loan.

## Import Libraries

In [209]:
import pandas as pd
import numpy as np
import os
import random
from collections import Counter

## Load the Data
Let us begin by reading the Loan Default dataset.

The original csv can be found at: https://www.kaggle.com/kmldas/loan-default-prediction

In [210]:
file_path = os.path.join("./", "Default_Fin.csv")
df = pd.read_csv(file_path)

In [211]:
# Features: Employed (1 = yes, 0 = no), Bank Balance, Annual Salary
# Classification: Defaulted (1 = yes, 0 = no)
df

,Index,Employed,Bank Balance,Annual Salary,Defaulted?
0,1,1,8754.36,532339.56,0
1,2,0,9806.16,145273.56,0
2,3,1,12882.60,381205.68,0
3,4,1,6351.00,428453.88,0
4,5,1,9427.92,461562.00,0
...,...,...,...,...,...
9995,9996,1,8538.72,635908.56,0
9996,9997,1,9095.52,235928.64,0
9997,9998,1,10144.92,703633.92,0
9998,9999,1,18828.12,440029.32,0


We have 3 features (employment, bank balance, and annual salary) that can be used to classify whether the person defaults on their loan. 

To construct the tree, we will need a cost function to determine the best classifier. Since we are doing classification, rather than regression, we can use the Gini Score.

## Split Data into Train and Test
Split into test and train data: 70% train. The training data is sampled randomly, without replacement.

In [212]:
df_train = df.sample(frac=0.7, replace=False)
df_test = df.drop(df_train.index)

## Making a Decision Tree

### Cost Function: Gini Score

We use the function $$G = \sum{pk(1-pk)}$$ where $G$ is the gini score of splitting by a certain feature, $pk$ is the proportion of same class inputs within a group.

In [213]:
def gini_score(feature: str, data: pd.DataFrame, split_value):
    """
    This function computes the gini score for splitting data by the given feature.
    Given a split value, we split the data into a left and right branch. For 
    each branch, we count the number of each class in the data. We use these 
    counts to compute pk for each class, and then the gini score.

    If one of the branches is empty, then we compute the gini score only using the 
    non-empty branch. 
    """
    
    left_branch = data.loc[data[feature] < split_value]
    right_branch = data.loc[data[feature] >= split_value]

    left_0 = left_branch.loc[left_branch['Defaulted?'] == 0]['Defaulted?'].count()
    left_1 = left_branch.loc[left_branch['Defaulted?'] == 1]['Defaulted?'].count()

    right_0 = right_branch.loc[right_branch['Defaulted?'] == 0]['Defaulted?'].count()
    right_1 = right_branch.loc[right_branch['Defaulted?'] == 1]['Defaulted?'].count()
    
    # Left branch is empty, compute pk for each class using right branch only
    if(len(left_branch) == 0 and len(right_branch) > 0):
        # We compute pk w.r.t. the first class: 0 (did not default on loan)
        pk_0_right = right_0 / (right_0 + right_1)
        pk_0 = pk_0_right

        # We compute pk w.r.t. the second class: 1 (did default on loan)
        pk_1_right = right_1 / (right_0 + right_1)
        pk_1 = pk_1_right

        gini_score = (pk_0*(1-pk_0) + pk_1*(1-pk_1))
        return(gini_score)
    
    # Right branch is empty, compute pk for each class using left branch only
    elif(len(right_branch) == 0 and len(left_branch) > 0):
        # We compute pk w.r.t. the first class: 0 (did not default on loan)
        pk_0_left = left_0 / (left_0 + left_1)
        pk_0 = pk_0_left

        # We compute pk w.r.t. the secon class: 1 (did default on loan)
        pk_1_left = left_1 / (left_0 + left_1)
        pk_1 = pk_1_left

        gini_score = (pk_0*(1-pk_0) + pk_1*(1-pk_1))
        return(gini_score)

    # Neither branch is empty, compute pk for each class using both branches
    else:
        # We compute pk w.r.t. the first class: 0 (did not default on loan)
        pk_0_left = left_0 / (left_0 + left_1)
        pk_0_right = right_0 / (right_0 + right_1)
        pk_0 = np.average([pk_0_left, pk_0_right])

        # We compute pk w.r.t. the second class: 1 (did default on loan)
        pk_1_left = left_1 / (left_0 + left_1)
        pk_1_right = right_1 / (right_0 + right_1)
        pk_1 = np.average([pk_1_left, pk_1_right])

        gini_score = (pk_0*(1-pk_0) + pk_1*(1-pk_1))
        return(gini_score)


### Generating a Classification
To generate a classification at a leaf node, we find the most popular class in the sub-group of training data at that leaf node.

In [214]:
def pop_class(data):
    """
    This function groups data by each class and gets the counts.
    
    The function returns the most popular (highest count) class in data.
    """
    # For each class, the counts in each column are the same, so we 
    # just select the first column
    count_df = data.groupby(['Defaulted?']).count()['Index']
    return(count_df.idxmax())
        

### Creating the Tree

In [215]:
def make_tree(features_list, train, test):
    """
    We begin with a list of all the features available in the data. The possible 
    values we can use to split the data for each feature are the min, max, and each 
    quartile. The feature/split-value combination with the lowest gini score is 
    used to split the training data first. Based on the value of that feature in 
    our test data relative to the chosen split-value, we move into the corresponding 
    branch of the tree and recursively repeat. We continue until we have no more 
    features to split with. 

    Note: test is one row of test data
    """

    if len(features_list) == 0:
        return(pop_class(train))

    min_gini = 1
    best_feature = ""
    best_split = 0

    # Iterate through each feature and split-value combination, find the 
    # combination that yields the lowest gini score.
    for feature in features_list:
        possible_splits = [
            train[feature].quantile(0),
            train[feature].quantile(0.25), 
            train[feature].quantile(0.5),
            train[feature].quantile(0.75),
            train[feature].quantile(1)
        ]
        for split in possible_splits:
            if (gini_score(feature, train, split) <= min_gini):
                min_gini = gini_score(feature, train, split)
                best_feature = feature
                best_split = split

    # We have now used up this classifier, remove it from the feature list
    features_list.remove(best_feature)
    

    split_value = best_split

    # Using the best_feature, and its split value, compare it to the test data
    # to determine whether to go left or right in the tree
    # Split the data like normal, but if we move into a sub-branch of the training
    # data that is empty, then we move into the opposite sub-branch
    if test[best_feature] <= split_value and len(train.loc[train[best_feature] <= split_value]) > 0:
        return(make_tree(features_list, train.loc[train[best_feature] <= split_value], test))
    else:
        if (len(train.loc[train[best_feature] > split_value]) > 0):
            return(make_tree(features_list, train.loc[train[best_feature] > split_value], test))
        else:
            return(make_tree(features_list, train.loc[train[best_feature] <= split_value], test))

### Testing the Desicion Tree on One Row of Test Data

In [216]:
# Randomly select an index for a row of test data
rand_index = random.sample(list(df_test.index), 1)[0]

test_1 = df_test.loc[rand_index]
features_list = ['Employed', 'Bank Balance', 'Annual Salary']

# Output format: (Prediction, actual)
make_tree(features_list, df_train, test_1), test_1['Defaulted?']

(0, 0.0)

We <u>correctly</u> predicted that this person did not default on their loan! Let us expand.

### Testing the Decision Tree on All Test Data

In [217]:
correct_count = 0

# Count the number of correct classifications for all test data
for i in list(df_test.index):
    features_list = ['Employed', 'Bank Balance', 'Annual Salary']
    test = df_test.loc[i]
    if(make_tree(features_list, df_train, test) == test['Defaulted?']):
        correct_count += 1

accuracy = (correct_count / len(df_test)) * 100

print("Accuracy: %.2f%%" % accuracy)

Accuracy: 96.80%


**Our decision tree algorithm yields a classification accuracy of 96.80%.**

## Decision Tree to Random Forest

In [218]:
def random_forest(features_list, train, test, num_trees, sample_size):
    """
    This function generates 'num_trees' number of decision trees. Each tree 
    is created from a sub-group of training data by sampling randomly 
    (with replacement) 'sample_size' number of times. 

    Given the list of features, a sub-group of training data, and a single 
    test datapoint, each tree makes its classification. The most popular 
    classification made by all the trees is returned as the final 
    classification of the random forest.  
    """
    classifications = []

    for i in range(num_trees):
        classifications.append(make_tree(
            features_list, 
            train.sample(n=sample_size, replace=True), 
            test
        ))
    
    return(Counter(classifications).most_common(1)[0][0])

### Testing the Random Forest on All Test Data

Let us create our random forests such that each forest contains 3 decision trees, each of which are trained on 40 data points. We select a low number like 40 so that we do not overtrain our model.

In [219]:
correct_count = 0

# Count the number of correct classifications for all test data
for i in range(len(df_test)):
    features_list = ['Employed', 'Bank Balance', 'Annual Salary']
    if (random_forest(features_list, df_train, df_test.iloc[i], 3, 40) == df_test.iloc[i]['Defaulted?']):
        correct_count += 1

accuracy = (correct_count / len(df_test)) * 100

print("Accuracy: %.2f%%" % accuracy)

Accuracy: 96.80%


**Our random forest algorithm yields a classification accuracy of 96.80%.**

## Extensions

We can extend the implementation of this random forest model by including a pruning step. For instance, using reduced error pruning, we can remove each of the 3 features individually, checking the accuracy cost for each removal. Given that there is a small number of features available, pruning is not a crucial step for this specific application.